Resources:
 - 

In [1]:
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.tensorboard import SummaryWriter

In [2]:
from models import *

In [3]:
d_c = DeformConv(28, 28)

In [4]:
print(d_c)

DeformConv(
  (offset_net): Conv2d(28, 18, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (deform_conv): DeformConv2d(28, 28, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
)


In [5]:
net_ = Model()
x = torch.ones([1, 1, 28, 28])
net_(x)

tensor([[ 0.0702, -0.0019,  0.0386,  0.0121, -0.0670, -0.0550,  0.0443,  0.0633,
         -0.0153, -0.0361]], grad_fn=<AddmmBackward>)

In [33]:
def get_cifar10_tforms(rotation=0, translation=(0,0)):
    mean = np.array([0.4914, 0.4822, 0.4465])
    std = np.array([0.2470, 0.2435, 0.2616])

    train_transform = transforms.Compose([
        transforms.RandomAffine(rotation, translate=translation),
        transforms.RandomCrop(30, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])
    test_transform = transforms.Compose([
        transforms.RandomAffine(rotation, translate=translation),
        transforms.CenterCrop(30),
        transforms.ToTensor(),
        transforms.Normalize(mean, std),
    ])

    return train_transform, test_transform


In [34]:
def get_loaders(rotation=0, translation=(0,0), dataset=datasets.MNIST):
    if dataset == datasets.CIFAR10:
        train_tform, test_tform = get_cifar10_tforms(rotation=rotation, translation=translation)
    elif dataset == datasets.MNIST:
        train_tform = transforms.Compose([
                transforms.RandomAffine(rotation, translate=translation),
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])
        test_tform = transforms.Compose([
                transforms.RandomAffine(0, translate=translation),
                transforms.ToTensor(),
                transforms.Normalize((0.1307,), (0.3081,))
            ])

    else:
        raise RuntimeError("Unknown Dataset type!")


    train_loader = torch.utils.data.DataLoader(
        dataset(
            root='../../../data/', 
            train=True, 
            transform=train_tform
        ), 
        batch_size=64, 
        shuffle=True, 
        num_workers=4
    )
        
    test_loader = torch.utils.data.DataLoader(
        dataset(
            root='../../../data/', 
            train=False, 
            transform=test_tform
        ), 
        batch_size=64, 
        shuffle=True, 
        num_workers=4
    )

    return train_loader, test_loader

In [5]:
def test(model, loss_fn, test_loader):
    num_correct = 0
    avg_loss = 0
    for step, (data, targets) in enumerate(test_loader):
        data = data.cuda()
        targets = targets.cuda()

        with torch.no_grad():
            preds = model(data)

        loss = loss_fn(preds, targets)
        avg_loss += loss.item()

        _, pred_labels = torch.max(preds, dim=1)
        
        num_correct += pred_labels.eq(targets).sum().item()

    # Because used a sum, make sure to divide by the length of total number in dataset
    acc = num_correct/len(test_loader.dataset)
    avg_loss /= len(test_loader)

    return acc, avg_loss

In [6]:
def train(model, loss_fn, optimizer, train_loader, test_loader, writer, num_epochs, log_freq):
    batch_count = 0
    for epoch in range (num_epochs):
        tot_loss = 0
        for i, data in enumerate(train_loader):
            batch_count += 1
            inputs, labels = data
            inputs, labels = inputs.cuda(), labels.cuda()
            
            optimizer.zero_grad()

            pred = model(inputs)
            loss = loss_fn(pred, labels)
            loss.backward()
            
            optimizer.step()

            if i % log_freq == (log_freq - 1):
                writer.add_scalar('train_loss', loss.item(), global_step=batch_count)
        
        acc, avg_loss = test(model, loss_fn, test_loader)
        writer.add_scalar('test_loss', avg_loss, global_step=epoch)
        writer.add_scalar('test_acc', acc, global_step=epoch)


In [14]:
def train_model(model, run_name, train_loader, test_loader, inp_channels=1, log_freq=100):
    net = model(inp_channels).cuda()
    optimizer = optim.SGD(net.parameters(), lr = 0.001, momentum=0.9)
    loss_fn = nn.CrossEntropyLoss()
    writer = SummaryWriter('./runs/%s'%run_name)
    train(net, loss_fn, optimizer, train_loader, test_loader, writer, 10, log_freq)
    acc, _ = test(net, loss_fn, test_loader)
    print('Accuracy: %s'%(acc*100) + '%')

# MNIST

In [10]:
train_loader, test_loader = get_loaders()
train_model(Model, 'MNIST_normal', train_loader, test_loader)

Accuracy: 98.82%


In [11]:
train_loader, test_loader = get_loaders()
train_model(DeformedConvModel, 'MNIST_deformed', train_loader, test_loader)

Accuracy: 99.1%


In [12]:
train_loader, test_loader = get_loaders(rotation=120)
train_model(Model, 'rotated_MNIST_normal', train_loader, test_loader)

Accuracy: 95.78%


In [13]:
train_loader, test_loader = get_loaders(rotation=120)
train_model(DeformedConvModel, 'rotated_MNIST_deformed', train_loader, test_loader)

Accuracy: 97.1%


In [14]:
train_loader, test_loader = get_loaders(translation=(0.3, 0.3))
train_model(Model, 'translated_MNIST_normal', train_loader, test_loader)

Accuracy: 95.71%


In [15]:
train_loader, test_loader = get_loaders(translation=(0.3, 0.3))
train_model(DeformedConvModel, 'translated_MNIST_deformed', train_loader, test_loader)

Accuracy: 98.27%


In [16]:
train_loader, test_loader = get_loaders(rotation=120, translation=(0.3, 0.3))
train_model(Model, 'trans_rot_MNIST_normal', train_loader, test_loader)

Accuracy: 87.28%


In [17]:
train_loader, test_loader = get_loaders(rotation=120, translation=(0.3, 0.3))
train_model(DeformedConvModel, 'trans_rot_MNIST_deformed', train_loader, test_loader)

Accuracy: 94.03%


# CIFAR 10

In [12]:
print(model)

Model(
  (features): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU()
    (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2))
    (4): ReLU()
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1))
    (7): ReLU()
    (8): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (9): Conv2d(128, 128, kernel_size=(3, 3), stride=(2, 2))
    (10): ReLU()
    (11): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (12): AvgPool2d(kernel_size=4, stride=4, padding=0)
  )
  (classify): Linear(in_features=128, out_features=10, bias=True)
)


In [19]:
Model(inp_channels=3)(torch.ones(64, 3, 32, 32)).shape

torch.Size([64, 10])

In [16]:
train_loader, test_loader = get_loaders(dataset=datasets.CIFAR10)
train_model(Model, 'CIFAR10_normal', train_loader, test_loader, inp_channels=3, log_freq=500)

Accuracy: 67.36%


In [32]:
DeformedConvModel(inp_channels=1).features(torch.ones(64, 1, 30, 30)).shape

torch.Size([64, 128, 1, 1])

In [35]:
train_loader, test_loader = get_loaders(dataset=datasets.CIFAR10)
train_model(DeformedConvModel, 'CIFAR10_deformed', train_loader, test_loader, inp_channels=3, log_freq=500)

Accuracy: 68.52000000000001%


In [36]:
train_loader, test_loader = get_loaders(rotation=90, dataset=datasets.CIFAR10)
train_model(Model, 'rotated_CIFAR10_normal', train_loader, test_loader, inp_channels=3, log_freq=500)

Accuracy: 53.010000000000005%


In [37]:
train_loader, test_loader = get_loaders(rotation=90, dataset=datasets.CIFAR10)
train_model(DeformedConvModel, 'rotated_CIFAR10_deformed', train_loader, test_loader, inp_channels=3, log_freq=500)

Accuracy: 55.16%
